# Why I like Julia
by Victor Boussange

## Julia is expressive and understandable

- This allows to easily understand the functions that you are using

- This allows you to easily contribute to packages ➡️ makes you more useful in this world



In [ ]:
function RSP_betweenness_qweighted(W::SparseMatrixCSC,
    Z::AbstractMatrix,
    qˢ::AbstractVector,
    qᵗ::AbstractVector,
    targetnodes::AbstractVector)

    Zⁱ = inv.(Z)
    Zⁱ[.!isfinite.(Zⁱ)] .= floatmax(eltype(Z)) # To prevent Inf*0 later...

    qˢZⁱqᵗ = qˢ .* Zⁱ .* qᵗ'
    sumqˢ = sum(qˢ)
    for j in axes(Z, 2)
        qˢZⁱqᵗ[targetnodes[j], j] -=  sumqˢ * qᵗ[j] * Zⁱ[targetnodes[j], j]
    end

    ZqˢZⁱqᵗZt = (I - W)'\qˢZⁱqᵗ
    ZqˢZⁱqᵗZt .*= Z

    return sum(ZqˢZⁱqᵗZt, dims=2) # diag(Z * ZqˢZⁱqᵗ')
end

## Julia is interactive



- Amazing IDE with VS code and inline prompts

![](https://code.visualstudio.com/assets/docs/languages/julia/overview.png)



- Julia code can be **developed interactively** on a remote server. More on that later on.


## Community well organized

Documentation is consistent and pleasant to access

![](img/Optimization.jl.png)

## Julia has a built-in package manager

Julia provides a **built-in package** manager called `Pkg` for **managing packages and environments**. Users can create a new environment and add specific packages to it, and each environment has its own set of dependencies. Julia also allows users to switch between different environments easily.

<div align="center"><img src="https://pkgdocs.julialang.org/v1/assets/logo.png" width="400"></img> </div>

More on that later on!

## Julia makes you productive



- It is easy to write generic code (with a bit of luck, a certain piece of Julia code can be directly ported to GPUs!)

- Research script can be easily transformed into **packages**, directly available to the community

### [EvoId.jl](https://github.com/vboussange/EvoId.jl)

A user friendly package aimed at simulating the evolutionary dynamics of a population structured over a complex spatio-evolutionary structures.

<div align="center"><img src="https://github.com/vboussange/EvoId.jl/raw/master/docs/src/assets/gif_evoid.gif" width="400"></img> </div>

### [HighDimPDE.jl](https://github.com/SciML/HighDimPDE.jl)
Solver for **highly dimensional, non-local, nonlinear PDEs** of the form.

### [PiecewiseInference.jl](https://github.com/vboussange/PiecewiseInference.jl)
Suite for parameter inference and model selection with dynamical models characterised by complex dynamics.
<div align="center"><img src="https://github.com/vboussange/PiecewiseInference.jl/raw/main/docs/animated.gif
" width="400"></img> </div>

## Execution speed!



Julia is great for fast simulations (differential equations, agent-based models, etc...)



#### `DifferentialEquations.jl` 
is one of the best library that exists for solving differential equations.

Here is a benchmark where the task consists of solving a certain differential equation model

![](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/figures/wrapper_packages_2_1.png)

#### Resources
- [Julia micro-benchmarks](https://julialang.org/benchmarks/)
- [SciML benchmarks](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/ode_wrapper_packages/)

## Composability



Packages can be composed at ease.


In [2]:
using OrdinaryDiffEq, Measurements

#Half-life of Carbon-14 is 5730 years.
c = 5.730 ± 2

#Setup
u0 = 1.0 ± 0.1
tspan = (0.0, 1.0)

#Define the problem
radioactivedecay(u,p,t) = -c*u

#Pass to solver
prob = ODEProblem(radioactivedecay,u0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
sol.u

48-element Vector{Measurement{Float64}}:
    1.0 ± 0.1
  0.968 ± 0.097
  0.932 ± 0.096
  0.888 ± 0.096
  0.842 ± 0.098
   0.79 ± 0.1
   0.74 ± 0.11
   0.69 ± 0.11
   0.64 ± 0.12
   0.59 ± 0.12
        ⋮
  0.015 ± 0.022
  0.013 ± 0.019
  0.011 ± 0.017
 0.0089 ± 0.015
 0.0073 ± 0.013
  0.006 ± 0.011
 0.0049 ± 0.0091
  0.004 ± 0.0077
 0.0032 ± 0.0065

## Automatic differentiation



- **Julia is automatic differentiation pervasive**
- You can differentiate any function written in Julia, which is very useful **model calibration and Bayesian inference** (and for Deep Learning).

```julia
using ForwarDiff

my_custom_model(p) = ...
ForwardDiff.derivative(my_custom_model, p)
```



In other languages, you need to write your model in a specific library to differentiate it (Stan, JAX, Pytorch)...
In Julia, 
*Write code first, choose AD second (and maybe adapt code).*

### More on AD
- https://gdalle.github.io/AutodiffTutorial/

- > *Differentiable Programming for Differential Equations: A Review*, Sapienza et al. (2024) [arXiv](https://arxiv.org/abs/2406.09699) 


### Applications of AD and composability



#### [Parametrizing a process in a process-based model with a neural network](ode_solvers_with_DL.jl)

```julia
function lotka_volterra(u, p, t)
    weights_nn, α, δ = p
    🐰, 🦊 = u

    nn_res = neuralnet(🦊, 🐰, weights_nn)

    d🐰 = α * 🐰 - nn_res[1]
    d🦊 = nn_res[2]  - δ * 🦊

    return [d🐰, d🦊]
end
```

#### Bayesian inference framework with deep learning

[Here is another cool example](https://turinglang.org/v0.24/tutorials/03-bayesian-neural-network/) of composability between [`Turing.jl`](https://turinglang.org/v0.24/) and `Flux.jl`

![](https://turinglang.org/v0.24/tutorials/figures/03_bayesian-neural-network_9_1.png)

## Caveat: trial-and-error development process

Julia is still in a trial and development process. You may expect some bugs in the libraries from time to time, or some rapid changes in syntax.